In [1]:
#! pip install geemap

In [72]:
import ee
import geemap

In [73]:
# If you are working with a shape file stored in google drive remove the quotes (""" """)

"""
#connect with drive
from google.colab import drive
drive.mount('/content/drive')
"""

"\n#connect with drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [74]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWjVHIT2EXgC8xTN33lKe9qs5bM514ZZLPGdugqkQIc29nei4P082UQ

Successfully saved authorization token.


In [75]:
Map = geemap.Map()
#Map

In [76]:
# https://sites.google.com/site/seriescol/shapes
shape = ee.FeatureCollection('users/dsrestrepo/mpio')

# Cauca = shape.filter(ee.Filter.eq('DPTO_CCDGO', 19))
# Popayan = shape.filter(ee.Filter.eq('MPIO_CCNCT	', 19001))
# Medellin = ee.FeatureCollection('users/dsrestrepo/Medellin')

Popayan = shape.filter(ee.Filter.eq('MPIOS', '19001'))

Map.addLayer(Popayan, {}, "Popayan")

In [77]:
years = ee.List.sequence(2013, 2020)
years.getInfo()

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [78]:
def yearly_image(year):
    
    start_date = ee.Date.fromYMD(year, 1, 1) 
    end_date = start_date.advance(1, "year")
    
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
        .filterDate(start_date, end_date) \
        .filterBounds(Popayan) 
    
    image =  ee.Algorithms.Landsat.simpleComposite(collection).clipToCollection(Popayan)

    return image

In [79]:
images = years.map(yearly_image)

In [83]:
vis_params = {'bands': ['B2',  'B3',  'B4'],
             'max': 128}

In [84]:
for index in range(0, 8):
    image = ee.Image(images.get(index))
    layer_name = "Image " + str(index + 2013)


    # Export the image, specifying scale and region.
    # https://developers.google.com/earth-engine/guides/reducers_reduce_region
    
    #geemap.ee_export_image_to_drive(image, description='landsat' + str(index + 2013)+'_geemap', 
    #                                folder='GEE/Popayan', 
    #                                region=Popayan.geometry(), 
    #                                scale=30)

    #image_unmask = image.clip(Popayan).unmask()
    #geemap.ee_export_image(image_unmask, filename='Images/'+'Landsat_Popayan_' + str(index + 2013) + '.tif', scale=27, region=Popayan.geometry(), file_per_band=False)
    #geemap.get_image_thumbnail(image_unmask, 'Images/'+'Landsat_PopayanUnmask_' + str(index + 2013) + '.png', vis_params, dimensions=500, format='png', region=Popayan.geometry())
    #geemap.get_image_thumbnail(image_unmask, 'Images/'+'Landsat_PopayanUnmask_' + str(index + 2013) + '.jpg', vis_params, dimensions=500, format='jpg', region=Popayan.geometry())
    #geemap.get_image_thumbnail(image, 'Images/'+'Landsat_PopayanUnmask_' + str(index + 2013) + '.png', vis_params, dimensions=500, format='png', region=Popayan.geometry())
    
    #task = ee.batch.Export.image.toDrive(**{
    #    'image': image,
    #    'description': 'lANDSAT_Popayan_' + str(index + 2013),
    #    'folder':'GEE/Popayan',
    #    'scale': 30,
    #    'region': Popayan.geometry()
    #})
    #task.start()
    
    Map.addLayer(image, vis_params, layer_name)

Map.center_object(Popayan, 13)
Map

Map(bottom=2068700.0, center=[2.476705214936409, -76.64337627568842], controls=(WidgetControl(options=['positi…

In [82]:
# geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=10)